<a href="https://colab.research.google.com/github/NishaMDev/SMS-Spam-Detection-with-Flask-Deployment/blob/main/SMS_Spam_Collection_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SMS Spam Collection Dataset

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"

In [3]:
%cd /content/gdrive/MyDrive/Kaggle

/content/gdrive/MyDrive/Kaggle


In [4]:
!kaggle datasets download -d uciml/sms-spam-collection-dataset

  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 21.7MB/s]


In [5]:
#!unzip \*.zip  && rm *.zip

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [7]:
df = pd.read_csv('spam.csv', encoding="latin-1")
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [8]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

In [9]:
df['label'] = df['v1'].map({'ham': 0, 'spam': 1})

In [10]:
X = df['v2']
y = df['label']

In [11]:
cv = CountVectorizer()
X = cv.fit_transform(X) # Fit the Data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [13]:
#Naive Bayes Classifier
clf = MultinomialNB()
clf.fit(X_train,y_train)

MultinomialNB()

In [14]:
clf.score(X_test,y_test)

0.9793365959760739

In [15]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1587
           1       0.93      0.92      0.92       252

    accuracy                           0.98      1839
   macro avg       0.96      0.95      0.96      1839
weighted avg       0.98      0.98      0.98      1839



Naive Bayes classifier is easy to implement but also provides very good result.

After training the model, it is desirable to have a way to persist the model for future use without having to retrain. To achieve this, we add the following lines to save our model as a .pkl file for the later use.

In [16]:
import joblib

joblib.dump(clf, 'NB_spam_model.pkl')

['NB_spam_model.pkl']

load and use saved model later like so:

In [17]:
NB_spam_model = open('NB_spam_model.pkl','rb')
clf = joblib.load(NB_spam_model)

In [18]:
!pip install flask-ngrok
!pip install flask-bootstrap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
%%writefile app.py
from flask_ngrok import run_with_ngrok
from flask import Flask,render_template,url_for,request
import pandas as pd 
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib
from pyngrok import ngrok


app = Flask(__name__)

port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url


@app.route('/')
def home():
	return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():
	df= pd.read_csv("spam.csv", encoding="latin-1")
	df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
	# Features and Labels
	df['label'] = df['v1'].map({'ham': 0, 'spam': 1})
	X = df['v2']
	y = df['label']
	
	# Extract Feature With CountVectorizer
	cv = CountVectorizer()
	X = cv.fit_transform(X) # Fit the Data
	from sklearn.model_selection import train_test_split
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
	#Naive Bayes Classifier
	from sklearn.naive_bayes import MultinomialNB

	clf = MultinomialNB()
	clf.fit(X_train,y_train)
	clf.score(X_test,y_test)
	#Alternative Usage of Saved Model
	# joblib.dump(clf, 'NB_spam_model.pkl')
	# NB_spam_model = open('NB_spam_model.pkl','rb')
	# clf = joblib.load(NB_spam_model)

	if request.method == 'POST':
		message = request.form['message']
		data = [message]
		vect = cv.transform(data).toarray()
		my_prediction = clf.predict(vect)
	return render_template('result.html',prediction = my_prediction)



if __name__ == '__main__':
	app.run()

Overwriting app.py


In [29]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=b8414d909064a077b2c805b9766529a4da1852ddb4112f116faa2ddc8c177c3c
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [24]:
!sudo cp ngrok /usr/local/bin    

In [32]:
!ngrok  authtoken 29pCLGOIl0P0sT5jUkiFOF5WPRd_MDgEW71rNfzZF66z9CDD

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [35]:
!python app.py

 * ngrok tunnel "http://5b87-34-74-120-223.ngrok.io" -> "http://127.0.0.1:5000"
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2022 06:54:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 06:54:28] "GET /static/css/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [29/May/2022 06:54:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/May/2022 06:54:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 06:54:29] "GET /static/css/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [29/May/2022 06:54:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 06:54:36] "GET /static/css/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [29/May/2022 06:54:37] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [29/May/2022 06:55:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 06:55:29] "GET /static/css/styles.css HTTP/1.1" 404 -
^C
